In [1]:
#import os
#import glob
from pathlib import Path
import yaml
import re

import logging

import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import train_test_split, KFold

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import pyarrow.parquet as pq
import pyarrow as pa


import hashlib


# Custom Utilities Module
from utils.paths import get_paths
from utils.file_io import load_data, save_data


# Show more columns
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 200)

# Initiate Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
# Get Path's Object
paths = get_paths()

logger.info(f"Project Root Path Loaded: {paths.root}")    
    
logger.info(f"Project Data Path Loaded: {paths.data}")

logger.info(f"Data Bronze Path Loaded: {paths.data_bronze}")

logger.info(f"Data Bronze Training Path Loaded: {paths.data_bronze_train}")
logger.info(f"Data Bronze Testing Path Loaded: {paths.data_bronze_test}")

logger.info(f"Data Silver Path Loaded: {paths.data_silver}")

logger.info(f"Data Silver Training Path Loaded: {paths.data_silver_train}")
logger.info(f"Data Silver Testing Path Loaded: {paths.data_silver_test}")

INFO:__main__:Project Root Path Loaded: /workspace
INFO:__main__:Project Data Path Loaded: /workspace/data
INFO:__main__:Data Bronze Path Loaded: /workspace/data/bronze
INFO:__main__:Data Bronze Training Path Loaded: /workspace/data/bronze/train
INFO:__main__:Data Bronze Testing Path Loaded: /workspace/data/bronze/test
INFO:__main__:Data Silver Path Loaded: /workspace/data/silver
INFO:__main__:Data Silver Training Path Loaded: /workspace/data/silver/train
INFO:__main__:Data Silver Testing Path Loaded: /workspace/data/silver/test


In [ ]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [3]:
BRONZE_META_COLS = [
    "meta__dataset",
    "meta__split",
    "meta__run_id",
    "meta__label_type",
    "meta__ingested_at_utc",
    "meta__source_file",
    "meta__source_row_id",
    "meta__record_id",
]


SILVER_META_COLS_ADDED = [
    "meta__label_source",
    "meta__layer",
    "meta__processed_at",
    "meta__cleaning_recipe",
    "meta__feature_set",
    "meta__silver_version",
]

In [ ]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [4]:
# Debugging Cell
# Verifies Parquet file headers and footer and true size of the file

p = paths.data_bronze_train / "pump_train_bronze.parquet"
print("Exists:", p.exists(), "Size (GB):", round(p.stat().st_size / (1024**3), 2))

with open(p, "rb") as f:
    head = f.read(4)
    f.seek(-4, 2)
    tail = f.read(4)

print("Header:", head)
print("Footer:", tail)

Exists: True Size (GB): 0.05
Header: b'PAR1'
Footer: b'PAR1'


In [5]:
# Debugging Cell
# Captures the row groups, number of rows, and column names from the parquet file

pf = pq.ParquetFile(paths.data_bronze_train / "pump_train_bronze.parquet")
print("Row groups:", pf.num_row_groups)
print("Rows:", pf.metadata.num_rows)
print("Columns:", pf.schema_arrow.names[:20], "...")

Row groups: 1
Rows: 220320
Columns: ['meta__dataset', 'meta__split', 'meta__run_id', 'meta__label_type', 'meta__ingested_at_utc', 'meta__source_file', 'meta__source_row_id', 'meta__record_id', 'Unnamed: 0', 'timestamp', 'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09'] ...


In [6]:
# Debugging Cell
# Loads only the meta columns from the parquet file and read/prints out the 

df_meta = pd.read_parquet(
    paths.data_bronze_train / "pump_train_bronze.parquet",
    columns=BRONZE_META_COLS,   # key: only a few columns
    engine="pyarrow"
)
df_meta.shape, df_meta.head()


((220320, 8),
   meta__dataset meta__split meta__run_id meta__label_type            meta__ingested_at_utc meta__source_file  meta__source_row_id       meta__record_id
 0          PUMP     unsplit      run_000             None 2025-12-27 23:21:07.737431+00:00        sensor.csv                    0  14598431322315673869
 1          PUMP     unsplit      run_000             None 2025-12-27 23:21:07.737431+00:00        sensor.csv                    1  15954729095895098000
 2          PUMP     unsplit      run_000             None 2025-12-27 23:21:07.737431+00:00        sensor.csv                    2  10041703297090838359
 3          PUMP     unsplit      run_000             None 2025-12-27 23:21:07.737431+00:00        sensor.csv                    3   2072036352569063261
 4          PUMP     unsplit      run_000             None 2025-12-27 23:21:07.737431+00:00        sensor.csv                    4   4365040424004714369)

In [7]:
pf = pq.ParquetFile(paths.data_bronze_train/"pump_train_bronze.parquet")
print("Row groups:", pf.num_row_groups, "Rows:", pf.metadata.num_rows)

Row groups: 1 Rows: 220320


In [ ]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [8]:
# Load Data
# dataframe = load_data(paths.data_bronze, "pump_train_bronze.parquet")

dataframe = load_data(paths.data_bronze_train, "pump_train_bronze.parquet")

INFO:utils.file_io:Loading Parquet: /workspace/data/bronze/train/pump_train_bronze.parquet


In [ ]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [9]:
# Add Layer Tag
dataframe["meta__layer"] = "silver"

# Add Silver Processing Column and timestamp start
dataframe["meta__processed_at"] = pd.Timestamp.now(tz="UTC")

# Add Silver Version Meta Column
dataframe["meta__silver_version"] = "v1"


In [10]:
TS_COLUMN = "timestamp"
STATUS_COLUMN = "machine_status"
LABEL_COLUMN = "anomaly_flag"

NORMAL_STATUS_VALUES = {"normal", "ok", "healthy", "running"}

_UNNAMED_RE = re.compile(r"^unnamed:\s*\d+(\.\d+)?$", flags=re.IGNORECASE)

In [ ]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [11]:
# Basic Dataframe Information/Summary

print("\nDataframe Information:")
dataframe.info()

print("Shape:", dataframe.shape)
print("\nData types:")
print(dataframe.dtypes)

print("\nMemory usage (MB):")
print(dataframe.memory_usage(deep=True).sum() / (1024 ** 2))

print("\nFirst 15 rows:")
display(dataframe.head(15))

print("\nBasic numeric summary:")
display(dataframe.describe().T)

print("\nBasic object / categorical summary:")
display(dataframe.describe(include="object").T)


Dataframe Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220320 entries, 0 to 220319
Data columns (total 66 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   meta__dataset          220320 non-null  category           
 1   meta__split            220320 non-null  category           
 2   meta__run_id           220320 non-null  object             
 3   meta__label_type       0 non-null       object             
 4   meta__ingested_at_utc  220320 non-null  datetime64[us, UTC]
 5   meta__source_file      220320 non-null  string             
 6   meta__source_row_id    220320 non-null  int64              
 7   meta__record_id        220320 non-null  uint64             
 8   Unnamed: 0             220320 non-null  int64              
 9   timestamp              220320 non-null  object             
 10  sensor_00              210112 non-null  float64            
 11  sensor_01      

,meta__dataset,meta__split,meta__run_id,meta__label_type,meta__ingested_at_utc,meta__source_file,meta__source_row_id,meta__record_id,Unnamed: 0,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,sensor_40,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status,meta__layer,meta__processed_at,meta__silver_version
0,PUMP,unsplit,run_000,None,2025-12-27 23:21:07.737431+00:00,sensor.csv,0,14598431322315673869,0,2018-04-01 00:00:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,NaN,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL,silver,2025-12-28 03:57:55.104592+00:00,v1
1,PUMP,unsplit,run_000,None,2025-12-27 23:21:07.737431+00:00,sensor.csv,1,15954729095895098000,1,2018-04-01 00:01:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,NaN,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL,silver,2025-12-28 03:57:55.104592+00:00,v1
2,PUMP,unsplit,run_000,None,2025-12-27 23:21:07.737431+00:00,sensor.csv,2,10041703297090838359,2,2018-04-01 00:02:00,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013,37.86777,48.17723,32.08894,1.708474,420.8480,NaN,462.7798,459.6364,2.500062,666.2234,399.9418,880.4237,501.3617,982.7342,631.1326,740.8031,849.8997,454.2390,778.5734,715.6266,661.5740,721.8750,694.7721,441.2635,169.9820,343.1955,200.9694,93.90508,41.40625,31.25000,69.53125,30.468750,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,NORMAL,silver,2025-12-28 03:57:55.104592+00:00,v1
3,PUMP,unsplit,run_000,None,2025-12-27 23:21:07.737431+00:00,sensor.csv,3,2072036352569063261,3,2018-04-01 00:03:00,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247,38.57977,48.65607,31.67221,1.579427,420.7494,NaN,462.8980,460.8858,2.509521,666.0114,399.1046,878.8917,499.0430,977.7520,625.4076,739.2722,847.7579,474.8731,779.5091,690.4011,686.1111,754.6875,683.3831,446.2493,166.4987,343.9586,193.1689,101.04060,41.92708,31.51042,72.13541,30.468750,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,NORMAL,silver,2025-12-28 03:57:55.104592+00:00,v1
4,PUMP,unsplit,run_000,None,2025-12-27 23:21:07.737431+00:00,sensor.csv,4,4365040424004714369,4,2018-04-01 00:04:00,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247,39.48939,49.06298,31.95202,1.683831,419.8926,NaN,461.4906,468.2206,2.604785,663.2111,400.5426,882.5874,498.5383,979.5755,627.1830,737.6033,846.9182,408.8159,785.2307,704.6937,631.4814,766.1458,702.4431,433.9081,164.7498,339.9630,193.8770,101.70380,42.70833,31.51042,76.82291,30.989580,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,NORMAL,silver,2025-12-28 03:57:55.104592+00:00,v1
5,PUMP,unsplit,run_000,None,2025-12-27 23:21:07.737431+00:00,sensor.csv,5,137312070346572


Basic numeric summary:


,count,mean,std,min,25%,50%,75%,max
meta__source_row_id,220320.0,1.101595e+05,6.360105e+04,0.000000e+00,5.507975e+04,1.101595e+05,1.652392e+05,2.203190e+05
meta__record_id,220320.0,9.224243e+18,5.322367e+18,1.873763e+14,4.610703e+18,9.220552e+18,1.381850e+19,1.844674e+19
Unnamed: 0,220320.0,1.101595e+05,6.360105e+04,0.000000e+00,5.507975e+04,1.101595e+05,1.652392e+05,2.203190e+05
sensor_00,210112.0,2.372221e+00,4.122274e-01,0.000000e+00,2.438831e+00,2.456539e+00,2.499826e+00,2.549016e+00
sensor_01,219951.0,4.759161e+01,3.296666e+00,0.000000e+00,4.631076e+01,4.813368e+01,4.947916e+01,5.672743e+01
sensor_02,220301.0,5.086739e+01,3.666820e+00,3.315972e+01,5.039062e+01,5.164930e+01,5.277777e+01,5.603299e+01
sensor_03,220301.0,4.375248e+01,2.418887e+00,3.164062e+01,4.283854e+01,4.422743e+01,4.531250e+01,4.822049e+01
sensor_04,220301.0,5.906739e+02,1.440239e+02,2.798032e+00,6.266204e+02,6.326389e+02,6.376157e+02,8.000000e+02
sensor_05,220301.0,7.339641e+01,1.729825e+01,0.000000e+00,6.997626e+01,7.557679e+01,8.091215e+01,9.999988e+01
sensor_06,215522.0,1.350154e+01,2.163736e+00,1.446759e-02,1.334635e+01,1.364294e+01,1.453993e+01,2.225116e+01



Basic object / categorical summary:


,count,unique,top,freq
meta__run_id,220320,1,run_000,220320
meta__label_type,0,0,NaN,NaN
timestamp,220320,220320,2018-08-31 23:43:00,1
machine_status,220320,3,NORMAL,205836
meta__layer,220320,1,silver,220320
meta__silver_version,220320,1,v1,220320


In [ ]:

# Checks for Unnamed columns
#unnamed_columns = [column for column in dataframe.columns if str(column).strip().lower().startswith("unnamed")]

#print("Unnamed columns:", unnamed_columns)


Unnamed columns: ['Unnamed: 0']


In [ ]:

# Drops Fully Empty Unnamed Columns
#dataframe = pd.read_csv(path)

#unnamed = dataframe.columns.astype(str).str.match(r"^Unnamed")
#empty_unnamed = unnamed & dataframe.isna().all(axis=0).to_numpy()
#dataframe = dataframe.loc[:, ~empty_unnamed]

In [12]:

suspected_unnamed_columns = []

for column in dataframe.columns:

    name = "" if column is None else str(column).strip()

    if name == "":
        suspected_unnamed_columns.append(column)
        
    elif _UNNAMED_RE.match(name):
        suspected_unnamed_columns.append(column)



In [13]:
for column in suspected_unnamed_columns:
    print(f"{column}; Nunique: {dataframe[column].nunique(dropna=False)}; Nulls: {dataframe[column].isna().sum()}")
    print(f"Head: {dataframe[column].head(5).tolist()}")


Unnamed: 0; Nunique: 220320; Nulls: 0
Head: [0, 1, 2, 3, 4]


In [14]:
# Remove Unnamed Columns
dataframe = dataframe.drop(columns=suspected_unnamed_columns, errors="ignore")

In [15]:
remaining = [column for column in suspected_unnamed_columns if column in dataframe.columns]
assert not remaining, f"Unnamed/suspected columns still present: {remaining}"


In [16]:
def _fill_if_blank(dataframe, column, value):
    """
    Fill dataframe[column] with `value` where dataframe[column] is NA or empty/whitespace.
    If the column doesn't exist, create it.
    """
    if column not in dataframe.columns:
        dataframe[column] = value
        return

    s = dataframe[column]
    blank_mask = s.isna() | (s.astype(str).str.strip() == "")
    dataframe.loc[blank_mask, column] = value

In [ ]:
if LABEL_COLUMN in dataframe.columns:
    dataframe[LABEL_COLUMN] = pd.to_numeric(dataframe[LABEL_COLUMN], errors="coerce").fillna(0).astype("int8")
    label_source_value = f"existing:{LABEL_COLUMN}"

elif STATUS_COLUMN in dataframe.columns:
    status = dataframe[STATUS_COLUMN].astype(str).str.strip().str.lower()
    dataframe[LABEL_COLUMN] = (~status.isin(NORMAL_STATUS_VALUES)).astype("int8")
    label_source_value = f"pump.{STATUS_COLUMN}: status not in {sorted(list(NORMAL_STATUS_VALUES))}"

else: 
    dataframe[LABEL_COLUMN] = 0
    label_source_value = "fallback:no_status_no_label"

_fill_if_blank(dataframe, "meta__label_type", LABEL_COLUMN)
_fill_if_blank(dataframe, "meta__label_source", label_source_value)


In [19]:
dataframe["anomaly_flag"].value_counts()

anomaly_flag
0    205836
1     14484
Name: count, dtype: int64

In [20]:
non_meta_columns = [column for column in dataframe.columns if column not in (BRONZE_META_COLS + SILVER_META_COLS_ADDED) ]

counts_dataframe = dataframe.loc[:, non_meta_columns]

unique_counts = counts_dataframe.nunique().sort_values(ascending=False)
display(unique_counts.to_frame("n_unique"))

high_card_cols = unique_counts[unique_counts > 50].index.tolist()
low_card_cols = unique_counts[unique_counts <= 50].index.tolist()

print("High-cardinality columns (>50 uniques):", high_card_cols)
print("Low-cardinality columns (<=50 uniques):", low_card_cols)

,n_unique
timestamp,220320
sensor_32,205414
sensor_34,203895
sensor_27,203198
sensor_29,201908
sensor_35,201781
sensor_36,201437
sensor_33,200855
sensor_10,198804
sensor_11,196368


High-cardinality columns (>50 uniques): ['timestamp', 'sensor_32', 'sensor_34', 'sensor_27', 'sensor_29', 'sensor_35', 'sensor_36', 'sensor_33', 'sensor_10', 'sensor_11', 'sensor_13', 'sensor_05', 'sensor_28', 'sensor_37', 'sensor_12', 'sensor_26', 'sensor_25', 'sensor_18', 'sensor_17', 'sensor_24', 'sensor_21', 'sensor_22', 'sensor_23', 'sensor_16', 'sensor_19', 'sensor_14', 'sensor_20', 'sensor_04', 'sensor_31', 'sensor_30', 'sensor_48', 'sensor_00', 'sensor_50', 'sensor_51', 'sensor_39', 'sensor_40', 'sensor_46', 'sensor_02', 'sensor_01', 'sensor_49', 'sensor_06', 'sensor_43', 'sensor_42', 'sensor_45', 'sensor_44', 'sensor_08', 'sensor_47', 'sensor_38', 'sensor_41', 'sensor_03', 'sensor_09', 'sensor_07']
Low-cardinality columns (<=50 uniques): ['machine_status', 'anomaly_flag', 'sensor_15']


In [21]:
# Duplicates
n_duplicates = counts_dataframe.duplicated().sum()
print(f"Number of duplicated rows: {n_duplicates}")

# Nulls
null_counts = counts_dataframe.isna().sum()
null_pct = (null_counts / len(counts_dataframe)) * 100

null_df = pd.DataFrame({"null_count": null_counts, "null_pct": null_pct})
null_df = null_df[null_df["null_count"] > 0].sort_values("null_pct", ascending=False)

print("\nColumns with missing values:")
display(null_df)


# Optional: drop full-duplicate rows
#if n_duplicates > 0:
#    dataframe = dataframe.drop_duplicates().reset_index(drop=True)
#   print(f"\nDropped {n_duplicates} duplicate rows. New shape: {dataframe.shape}")



#
print("Shape:", counts_dataframe.shape)
print("Full-row duplicates:", int(counts_dataframe.duplicated().sum()))


nulls = counts_dataframe.isna().sum().to_frame("null_count")
nulls["null_pct"] = (nulls["null_count"] / len(counts_dataframe) * 100).round(2)
display(nulls.sort_values("null_pct", ascending=False).head(65))

Number of duplicated rows: 0

Columns with missing values:


,null_count,null_pct
sensor_15,220320,100.000000
sensor_50,77017,34.956881
sensor_51,15383,6.982117
sensor_00,10208,4.633261
sensor_07,5451,2.474129
sensor_08,5107,2.317992
sensor_06,4798,2.177741
sensor_09,4595,2.085603
sensor_01,369,0.167484
sensor_30,261,0.118464


Shape: (220320, 55)
Full-row duplicates: 0


,null_count,null_pct
sensor_15,220320,100.00
sensor_50,77017,34.96
sensor_51,15383,6.98
sensor_00,10208,4.63
sensor_07,5451,2.47
sensor_08,5107,2.32
sensor_06,4798,2.18
sensor_09,4595,2.09
sensor_01,369,0.17
sensor_30,261,0.12


In [22]:
DROPPED_COLUMNS = []

MISSING_THRESHOLD = 0.90
MISSING_PERCENTAGE = dataframe.isna().mean()
COLUMNS_TO_DROP = MISSING_PERCENTAGE[MISSING_PERCENTAGE >= MISSING_THRESHOLD].index.tolist()

DROPPED_COLUMNS = COLUMNS_TO_DROP

dataframe = dataframe.drop(columns=COLUMNS_TO_DROP)

print("Dropped:", DROPPED_COLUMNS)

Dropped: ['sensor_15']


In [23]:
dataframe[TS_COLUMN].info()

<class 'pandas.core.series.Series'>
RangeIndex: 220320 entries, 0 to 220319
Series name: timestamp
Non-Null Count   Dtype 
--------------   ----- 
220320 non-null  object
dtypes: object(1)
memory usage: 1.7+ MB


In [24]:
def parse_timestamp(dataframe, TS_COLUMN):
    dataframe = dataframe.copy()
    if TS_COLUMN and TS_COLUMN in dataframe.columns:
        dataframe[TS_COLUMN] = pd.to_datetime(dataframe[TS_COLUMN], errors="coerce", utc=True)
    return dataframe

dataframe = parse_timestamp(dataframe, TS_COLUMN)



In [25]:
dataframe[TS_COLUMN].info()

<class 'pandas.core.series.Series'>
RangeIndex: 220320 entries, 0 to 220319
Series name: timestamp
Non-Null Count   Dtype              
--------------   -----              
220320 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1)
memory usage: 1.7 MB


In [26]:
def standardize_columns(dataframe):
    dataframe = dataframe.copy()
    dataframe.columns = (
        pd.Index(dataframe.columns)
        .astype(str)
        .str.strip()
        .str.lower()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace("-", "_")
    )
    return dataframe


In [27]:
dataframe = standardize_columns(dataframe)

In [28]:
numeric_cols = counts_dataframe.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = counts_dataframe.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

# Histograms for numeric columns
#for col in numeric_cols:
#    plt.figure()
#    sns.histplot(data=dataframe, x=col, kde=True)
#    plt.title(f"Distribution of {col}")
#    plt.tight_layout()
#    plt.show()


Numeric columns: ['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25', 'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30', 'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35', 'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40', 'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45', 'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49', 'sensor_50', 'sensor_51', 'anomaly_flag']
Categorical columns: ['timestamp', 'machine_status']


In [29]:
# Find Sensor Columns

SENSOR_COLUMNS = []

prefer_prefixes = "sensor_"    

LABEL_AND_TIME_COLUMNS = [TS_COLUMN, LABEL_COLUMN, STATUS_COLUMN]

exclude_list = [BRONZE_META_COLS + SILVER_META_COLS_ADDED + LABEL_AND_TIME_COLUMNS]

prefixed = []

prefix_filterd = []

for column in dataframe.columns:
    column_name = str(column)

    starts_with_any_prefix = False

    for prefix in prefer_prefixes:
        if column_name.startswith(prefix):
            starts_with_any_prefix = True
            break
    
    if starts_with_any_prefix:
        prefixed.append(column)

for column in prefixed:
    if column not in exclude_list:
        prefix_filterd.append(column)


prefixed = prefix_filterd


if len(prefixed) > 0:
    sorted(prefixed)


SENSOR_COLUMNS = prefixed

In [30]:
SENSOR_COLUMNS

['sensor_00',
 'sensor_01',
 'sensor_02',
 'sensor_03',
 'sensor_04',
 'sensor_05',
 'sensor_06',
 'sensor_07',
 'sensor_08',
 'sensor_09',
 'sensor_10',
 'sensor_11',
 'sensor_12',
 'sensor_13',
 'sensor_14',
 'sensor_16',
 'sensor_17',
 'sensor_18',
 'sensor_19',
 'sensor_20',
 'sensor_21',
 'sensor_22',
 'sensor_23',
 'sensor_24',
 'sensor_25',
 'sensor_26',
 'sensor_27',
 'sensor_28',
 'sensor_29',
 'sensor_30',
 'sensor_31',
 'sensor_32',
 'sensor_33',
 'sensor_34',
 'sensor_35',
 'sensor_36',
 'sensor_37',
 'sensor_38',
 'sensor_39',
 'sensor_40',
 'sensor_41',
 'sensor_42',
 'sensor_43',
 'sensor_44',
 'sensor_45',
 'sensor_46',
 'sensor_47',
 'sensor_48',
 'sensor_49',
 'sensor_50',
 'sensor_51']

In [31]:
def coerce_numeric(dataframe, columns):
    dataframe = dataframe.copy()
    for column in columns:
        if column in dataframe.columns:
            dataframe[column] = pd.to_numeric(dataframe[column], errors="coerce")
    return dataframe

dataframe = coerce_numeric(dataframe, SENSOR_COLUMNS)

In [32]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220320 entries, 0 to 220319
Data columns (total 66 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   meta__dataset          220320 non-null  category           
 1   meta__split            220320 non-null  category           
 2   meta__run_id           220320 non-null  object             
 3   meta__label_type       220320 non-null  object             
 4   meta__ingested_at_utc  220320 non-null  datetime64[us, UTC]
 5   meta__source_file      220320 non-null  string             
 6   meta__source_row_id    220320 non-null  int64              
 7   meta__record_id        220320 non-null  uint64             
 8   timestamp              220320 non-null  datetime64[ns, UTC]
 9   sensor_00              210112 non-null  float64            
 10  sensor_01              219951 non-null  float64            
 11  sensor_02              220301 non-null 

In [33]:
# 
dataframe["meta__asset_id"] = "pump_asset_001"

In [34]:
# Save Data as Parquet
save_data(dataframe, paths.data_silver_train, "pump_train_silver")

INFO:utils.file_io:Saving DataFrame to Parquet: /workspace/data/silver/train/pump_train_silver.parquet
INFO:utils.file_io:Saved: pump_train_silver.parquet | shape=(220320, 67) | columns=['meta__dataset', 'meta__split', 'meta__run_id', 'meta__label_type', 'meta__ingested_at_utc', 'meta__source_file', 'meta__source_row_id', 'meta__record_id', 'timestamp', 'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25', 'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30', 'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35', 'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40', 'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45', 'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49', 'sensor

PosixPath('/workspace/data/silver/train/pump_train_silver.parquet')